# Mining Tweet About BigData ?
### Collect the words people Tweet, when they Tweet about BigData
Find highest frequency words among Tweets about, for example, BigData in different locations.

## Project goals:
#### Project goal is to findinf most popular word in Tweet's about a given subject, and compare the results from different locations. I used BigData as an example, and added a political view analyzis as well :). In the same way, you can analize tweets about any subject.
#### The repository contais Hadoop MapReduce word count and top n words functionality, to process the collected words from Tweeter. Here we use Python Collection library, for demonstration.
### In this Python notebook, we:
* Implement Tweets retrieving, according to a given query.
* Reserch Tweet data structure.
* Reserch Tweets location issues.
* Implement word tokenizing and filtering.

In [1]:
import numpy as np
import pandas as pd
import tweepy   #Python library for accessing the Twitter API.

import nltk     #Python NLP library for tokenizing tweeter text
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\yoav
[nltk_data]     fridman\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## 1. Twitter API Authentication 
In order to extract tweets, we need an access kyes to Twitter API. To get those, you need a twitter acount, and you need to create a Twitter App to get the following 4 keys:

* Consumer Key (API Key)
* Consumer Secret (API Secret)
* Access Token
* Access Token Secret

Write those kyes in a text file, each in separate row. name the file: auth.k
Website to create the Twitter App https://apps.twitter.com/. 

In [2]:
# AUTHENTICATION (OAuth)
def tw_oauth():
    authfile = 'auth.k'
    with open(authfile, "r") as f:
        ak = f.readlines()
    f.close()
    auth1 = tweepy.auth.OAuthHandler(ak[0].replace("\n",""), ak[1].replace("\n",""))
    auth1.set_access_token(ak[2].replace("\n",""), ak[3].replace("\n",""))
    return tweepy.API(auth1)

## 2. Tweet data structure
Lets extract 5 recent tweets, and view the tweet data structure.

In [3]:
example_tweet = None

api = tw_oauth()
print ('5 recent tweets about Bigdata:')
Counter = 0
for tweet in tweepy.Cursor(api.search, q = 'Bigdata',lang = 'en',count = 5).items(): 
    print (tweet.text)
    Counter += 1
    if (Counter == 5):
        example_tweet = tweet 
        break
    

5 recent tweets about Bigdata:
RT @AiFinTek: AIRDROP Alert -Crypto project live, Free Tokens,100$ AFTK TOKEN,50$ AFTK Friend Referral, https://t.co/Hy9MZ5jfLS #ICO #block…
RT @jblefevre60: What are the #Top10 #Disruptive #technologies?

#fintech #Digital #Blockchain #AI #AR #VR #Drones #BigData #IoT #Robots #3…
RT @IainLJBrown: Accelerating big data for social good with UNICEF

Read more here: https://t.co/kMc9w7n63p

#BigData #DataScience #Machine…
RT @MarcoPark21: RT DeepLearn007 "RT DeepLearn007: Stanford University: Deep Learning Comes Full Circle
#AI #MachineLearning #DeepLearning…
How Is #BigData Influencing the Education Sector? #AI #ML  https://t.co/jMpsNVIWLE


** Tweet data is an object containing information about the authour and the Tweet.Lest view few fildes that is relevant to us.**

In [4]:
# We print info from the first tweet:
print('id: ', example_tweet.id)
print('created_at:', example_tweet.created_at)
print('geo:',example_tweet.geo)
print('coordinates',example_tweet.coordinates)

print('text:',example_tweet.text)                   #tweet text
print('geo_enabled:',example_tweet.author.geo_enabled)     #is author/user account geo enabled


id:  994303915171565568
created_at: 2018-05-09 19:51:41
geo: None
coordinates None
text: How Is #BigData Influencing the Education Sector? #AI #ML  https://t.co/jMpsNVIWLE
geo_enabled: True


## 3. Tokenize tweet
** Lest look at tweet test example:**<br>
RT @Hiredscore: Our team is gearing up for @Littler's 35th annual Executive Employer Conference in Phoenix, AZ this week! Attending the con…<br>
** To analize commom words in tweet, We need to tokenize and filter tweet text**<br>
* we change all words to lower case not to have doubles
* We will use nltk library for tokenizing
* we will remove stop words a, is, the...
* we will remove unintersting words, specific to tweeter, as: RT, &amp
* nltk tokenizing separate @ and # from the words, so if we limit word size to >1, its good enought.
* We will get user_id as a word in our list, since nltk splits @ronbon to @ and ronbon, but its interseting to the users that tweet so much about the given subgect.
    

In [5]:
# List of stop words, words we dont want to appear in our list.
s_words = {'ourselves', 'hers', 'between', 'yourself', 'but', 'again', 'there', 'about', 
              'once', 'during', 'out', 'very', 'having', 'with', 'they', 'own', 'an', 'be',
              'some', 'for', 'do', 'its', 'yours','de', 'vs', 'such', 'into', 'of', 'most', 'itself',
              'other', 'off', 'is', 's', 'am', 'or', 'who', 'as', 'from', 'him', 'each',
              'the', 'themselves', 'until', 'below', 'are', 'we', 'these', 'your', 'his', 
              'through', 'don', '\'\'','nor', 'me', 'were', 'her', 'more', 'himself',
              'this', 'down', 'should', 'our', 'their', 'while', 'above', 'both', 
              'up', 'to', 'ours', 'had', 'she', 'all', 'no', 'when', 'at', 'any', 
              'before', 'them', 'same', 'and', 'been', 'have', 'in', 'will', 'on',
              'does', 'yourselves', 'then', 'that', 'because', 'what', 'over', 'why',
              'so', 'can', 'did', 'not', 'now', 'under', 'he', 'you', 'herself', 'has',
              'just', 'where', 'too', 'only', 'myself', 'which', 'those', 'i', 'after',
              'few', 'whom', 't', 'being', 'if', 'theirs', 'my', 'against', 'a', 'by',
              'doing', 'it', 'how', 'further', 'was', 'here', 'than', 'The',
              'rt', '&amp', ' ', '','``', 'http', 'via', 'https', 'amp','\'s'}


In [6]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
def tokenize_tweet_text(tweet_text, Qye_words = None):
    
    word_tokens = word_tokenize(tweet_text)

    filtered_sentence = []

    for w in word_tokens:
        if w.lower() not in s_words | set(Qye_words):
            if (len(w)<= 1): continue
            if ('\\' in w): continue
            if ('/' in w): continue
                
            if (w.endswith('...')): w = w[:-4]
            if (w.startswith('...')): w = w[4:]
            if (w.endswith('-')): w = w[:-1]
            if (w.startswith('-')): w = w[1:]
            if (w.endswith('…')): w = w[:-1]
            if (w.startswith('…')): w = w[1:]
            if (w.isdigit()): continue
            filtered_sentence.append(w.lower())

    return filtered_sentence

In [7]:
# Lets check the tokenizing function

example_sent = "April 2018 JAX Magazine is Out: Machine Learning. #BigData #DeepLearning #MachineLeaning #DataScience #AI #Python #RStats @filtration \\ppo."
word_tokens = tokenize_tweet_text(example_sent, Qye_words = ['BigData'])

print(word_tokens)


['april', 'jax', 'magazine', 'machine', 'learning', 'bigdata', 'deeplearning', 'machineleaning', 'datascience', 'ai', 'python', 'rstats', 'filtration']


## 4. Retrive Tweet data definitions
There is a lot of data associated with each tweet. Lest view few the search fieldes that is relevant to our task.<br>
refernce: https://developer.twitter.com/en/docs/tweets/search/guides/standard-operators
* **q	required**	A UTF-8, URL-encoded search query of 500 characters maximum, including logical operators operators. <br>
    example: <br>
    * q = 'bigdata' : will retrive Tweets that have the word bigdata
    * q = 'puppy filter:media' : will retrive Tweets containing “puppy” and an image or video.
* **geocode	optional**	Returns tweets by users located within a given radius of the given latitude/longitude. The location is preferably taken from the Geotagging API, but will fall back to their Twitter profile. The parameter value is specified by ” latitude,longitude,radius ”, where radius units must be specified as either ” mi ” (miles) or ” km ” (kilometers). Note that you cannot use the near operator via the API to geocode arbitrary locations; however you can use this geocode parameter to search near geocodes directly. A maximum of 1,000 distinct “sub-regions” will be considered when using the radius modifier.<br>
    example: 37.781157 -122.398720 1mi
* **lang	optional**	Restricts tweets to the given language, given by an ISO 639-1 code. Language detection is best-effort. We will use only english<br>
    example: eu, en
* **count	optional**	The number of tweets to return per page, up to a maximum of 100. Defaults to 15. This was formerly the “rpp” parameter in the old Search API.<br>
    example: 100
<br>
Full definition can be found here: https://developer.twitter.com/en/docs/tweets/search/api-reference/get-search-tweets


## 4.1 How can we Filter Tweet by location


When working with Tweet data, there are two classes of geographical metadata:<br>

* **Tweet location** - Available when user shares location at time of Tweet.
* **Account Location** - Based on the ‘home’ location provided by user in their public profile. This is a free-form character field and may or may not contain metadata that can be geo-referenced.
Nullable . The user-defined location for this account’s profile. Not necessarily a location, nor machine-parseable. This field will occasionally be fuzzily interpreted by the Search service. Example: "location": "San Francisco, CA"

**Important Notes:**

    * Geographical coordinates are provided in the [LONG, LAT] order. <br>
The one exception is the deprecated  ‘geo’ attribute, which has the reverse [LAT, LONG] order.

    
#### Tweet locations ("geo-tagged" Tweets)
Twitter enables users to specify a location for individual Tweets. Tweet-specific location information falls into two general categories:

    * Tweets with a specific latitude/longitude “Point” coordinate
    * Tweets with a Twitter “Place” 
Tweets with a Point coordinate come from GPS enabled devices, and represent the exact GPS location of the Tweet in question. This type of location does not contain any contextual information about the GPS location being referenced (e.g. associated city, country, etc.), unless the exact location can be associated with a Twitter Place.

Tweets with a Twitter “Place” contain a polygon, consisting of 4 lon-lat coordinates that define the general area (the “Place”) from which the user is posting the Tweet. Additionally, the Place will have a display name, type (e.g. city, neighborhood), and country code corresponding to the country where the Place is located, among other fields.

#### Autor location 
Author location is an an arbitary centence that describe the author location. It is saved in authorloc varible. Author location for example can be:
* Hillingdon, London
* NY USA
Or any other centence the user chose to use as location description.
** To filter our desired locations, we will check if strings such as NY or London are contained in the authorlocation filed.

## Conclusions:
### From our tests and Tweeter documentation, not more then 0.5% of the tweets has geo tagging, so to get the full picture, we will filter tweet locations by the description in the Author location field.

## 4.2 Filter tweets by time
* **since**	optional filed in tweeter retriving. It Filter tweets created from the given date. Date should be formatted as YYYY-MM-DD. **Keep in mind that the search index has a 7-day limit**. In other words, no tweets will be found for a date older than one week!!.<br>
    example: 2015-07-19

source: https://developer.twitter.com/en/docs/tutorials/filtering-tweets-by-location <br>    

In [8]:
# Define start date of our word survey
import datetime
delta = -30
start_date = datetime.datetime.now() + datetime.timedelta(delta)
start_date = str(start_date.year) + '-' + str(start_date.month) + '-' + str(start_date.day)
print('Collecting tweets since: ', start_date)


In [11]:
print('Collecting tweets since: ', start_date)
# TWEEPY SEARCH FUNCTION
# refernce: https://developer.twitter.com/en/docs/tweets/search/api-reference/get-search-tweets
def tw_get_tweets(api,  query_in, Qye_words, geo, location, num_tweets=200):

    counter = 0
    example_tweet = None
    word_list = []
    
    # We will not use geocode inorder not to miss most of the tweets.
    for tweet in tweepy.Cursor(api.search,
                                q = query_in,           # the actual words we search
                                #geocode = geo,          # location
                                since = start_date,
                                count = num_tweets).items():

        #TWEET INFO
        created = tweet.created_at             #tweet created
        text    = tweet.text                   #tweet text
        tweet_id = tweet.id                    #tweet ID# (not author ID#)
        cords   = tweet.coordinates            #geographic co-ordinates
        geo_e   = tweet.author.geo_enabled     #is author/user account geo enabled?
        place   = tweet.place
        authorloc = tweet.author.location      #author/user location
        
        if any(word in authorloc for word in location):
            word_list += tokenize_tweet_text(text, Qye_words = Qye_words)

        counter = counter +1
        if (counter >= num_tweets):
            break
    return word_list

 ## 5. Lets Run some tests

In [12]:
import sys
import os
import os.path
from collections import Counter

authfile = 'auth.k'
api = tw_oauth()

df_results =  pd.DataFrame()
i = 0

Qye_words= ['bigdata']
query_in = 'BIGDATA+BigData+BigData'

# example of locations
location_list = ['NY', 'London', 'Mumbai', 'Paris']
query_list = [['NY', 'New York'], 
              ['London', 'london'],
              ['Mumbai', 'mumbai', 'bombay', 'Bombay'],
              ['paris', 'Paris']]

# lets view frequent words about Bigdata in different cities
for i in range(len(location_list)):
    
    location_words = query_list[i]

    print('Retreiving tweets since: ', start_date, ' about', Qye_words[0] ,
          ' Tweeted in locations: ',  location_list[i] )

    words = tw_get_tweets(api,  query_in, Qye_words = Qye_words,
                                      location = location_words,
                                      geo = None,
                                      num_tweets=2000)

    # lets count and sort words
    counts = Counter(words)
    word_list=[]
    freq_list=[]
    
    if (len(counts) > 0):
        for item, frequency in counts.most_common(10):
            word_list.append(item)
            freq_list.append(frequency)


        df_results[location_list[i] + '_words'] = word_list
        df_results[location_list[i] + '_freq'] = freq_list  

df_results.head(10)

Retreiving tweets since:  2018-4-9  about bigdata  Tweeted in locations:  NY
Retreiving tweets since:  2018-4-9  about bigdata  Tweeted in locations:  London
Retreiving tweets since:  2018-4-9  about bigdata  Tweeted in locations:  Mumbai
Retreiving tweets since:  2018-4-9  about bigdata  Tweeted in locations:  Paris


,NY_words,NY_freq,London_words,London_freq,Mumbai_words,Mumbai_freq,Paris_words,Paris_freq
0,iot,29,ai,33,machinelearning,2,data,22
1,ai,25,iot,25,kirkdborne,2,read,20
2,data,15,read,23,aftk,2,iainljbrown,19
3,cybersecurity,13,data,21,datascience,2,big,17
4,tech,13,iainljbrown,17,falling,1,datascience,17
5,digital,13,datascience,17,project,1,ai,16
6,fisher85m,11,machinelearning,16,love,1,machinelearning,15
7,vladobotsvadze,11,analytics,15,block,1,iot,12
8,analytics,10,google,12,short,1,ml,7
9,vr,10,vladobotsvadze,11,alert,1,artificialintelligence,6


## Lets query more intersting staff
Lest check the popular words in Tweets abount president Trump and Iran in different countries.

In [14]:
df_politics =  pd.DataFrame()
i = 0

# list of words that we don't want to count, because they are part of the query
# lower case is enought, because we filter words in lower case
Qye_words= ['trump', 'iran', 'president', 'us', 'america']

# build a query with logic operators
query_in ='(Trump AND Iran) OR (trump AND iran) OR (TRUMP AND IRAN)'

# define location names
location_list = ['US', 'INDIA', 'ENGLAND']

# define list of string by wich tweets are filtered
query_list = [['NY', 'New York', 'Washington', 'San Fransisco', 'US', 'USA'], 
              ['Mumbai', 'mumbai', 'bombay', 'Bombay', 'Delhi', 
               'India', 'Bangalore', 'INDIA'],
                ['London', 'london', 'Liverpool', 'England', 'Manchester']]


# lets view frequent words about Bigdata in different cities
for i in range(len(location_list)):
    
    location_words = query_list[i]

    print('Retreiving tweets since: ', start_date, ' about', Qye_words[0] ,
          ' Tweeted in locations: ',  location_list[i] )

    words = tw_get_tweets(api,  query_in, Qye_words = Qye_words,
                                      location = location_words,
                                      geo = None,
                                      num_tweets=6000)

    # lets count and sort words
    counts = Counter(words)
    word_list=[]
    freq_list=[]
    
    if (len(counts) > 0):
        for item, frequency in counts.most_common(10):
            word_list.append(item)
            freq_list.append(frequency)


        df_politics[location_list[i] + '_words'] = word_list
        df_politics[location_list[i] + '_freq'] = freq_list  

df_politics.head(10)

Retreiving tweets since:  2018-4-9  about trump  Tweeted in locations:  US
Retreiving tweets since:  2018-4-9  about trump  Tweeted in locations:  INDIA
Retreiving tweets since:  2018-4-9  about trump  Tweeted in locations:  ENGLAND


,US_words,US_freq,INDIA_words,INDIA_freq,ENGLAND_words,ENGLAND_freq
0,deal,436,deal,14,deal,82
1,nuclear,149,nuclear,4,nuclear,31
2,decision,80,syria,4,n't,19
3,obama,77,dow,3,speech,16
4,world,50,went,3,donald,15
5,go,50,breaks,3,fact,14
6,kurteichenwald,38,celebrate,3,buckle,14
7,went,37,facts,3,reckless,14
8,stocks,37,energy,3,checked,14
9,iranian,37,matter,3,nowthisnews,14
